> **Note:** This notebook is designed for Google Colab. Please ensure your data is in a `data` folder in your Google Drive. The notebook will mount your Drive and access the data from there.

# Raster Tile Mosaicing WorkflowThis notebook demonstrates the complete workflow for merging multiple georeferenced satellite raster tiles (GeoTIFFs) into a single, seamless, cloudless mosaic. The steps follow the requirements outlined in the task README.

## 1. Import Required LibrariesWe will use geospatial and scientific Python libraries such as rasterio, numpy, matplotlib, and glob for file handling and visualization.

In [ ]:
# Install required libraries if running in Colab/Kaggle (uncomment if needed)# !pip install rasterio matplotlib numpy scikit-imageimport osimport globimport rasteriofrom rasterio.merge import mergefrom rasterio.plot import showimport numpy as npimport matplotlib.pyplot as plt

## 2. Verify Data Folder ContentsCheck that the 'data' folder exists and is accessible.

In [ ]:
# Check if the 'data' folder existsDATA_DIR = 'data'if os.path.exists(DATA_DIR) and os.path.isdir(DATA_DIR):    print(f"'{DATA_DIR}' folder found.")else:    raise FileNotFoundError(f"'{DATA_DIR}' folder not found. Please check your extraction.")

## 3. List Extracted FilesList all files and directories inside the 'data' folder to confirm extraction and see available tiles.

In [ ]:
# List all files in the data directoryfiles = os.listdir(DATA_DIR)print(f"Files in '{DATA_DIR}':")for f in files:    print(f)

## 4. Preview a Sample Raster TileOpen and display basic information about a sample GeoTIFF file from the data folder.

In [ ]:
# Find a sample GeoTIFF file in the data foldersample_files = glob.glob(os.path.join(DATA_DIR, '*.tif'))if sample_files:    sample_file = sample_files[0]    print(f"Sample file: {sample_file}")    with rasterio.open(sample_file) as src:        print("CRS:", src.crs)        print("Shape:", src.shape)        print("Bounds:", src.bounds)        show(src.read(1), title="Sample Tile Band 1")else:    print("No GeoTIFF files found in the data folder.")

## 5. Load and Validate All TilesRead all GeoTIFF tiles, check their CRS, resolution, and spatial extent to ensure they can be mosaiced together.

In [ ]:
# Read all GeoTIFF files and validate their propertiessrc_files_to_mosaic = []crs_set = set()res_set = set()for fp in sample_files:    src = rasterio.open(fp)    src_files_to_mosaic.append(src)    crs_set.add(str(src.crs))    res_set.add(src.res)    print(f"{os.path.basename(fp)}: CRS={src.crs}, Resolution={src.res}, Shape={src.shape}")print(f"Unique CRS: {crs_set}")print(f"Unique Resolutions: {res_set}")if len(crs_set) > 1 or len(res_set) > 1:    print("Warning: Not all tiles have the same CRS or resolution. Reprojection/resampling may be needed.")

## 6. Cloud Masking (Remove Clouds from Tiles)To remove clouds, we will use a simple threshold-based approach on the brightness of the image. For more robust results, you can use cloud masks provided with the data or apply machine learning models. Here, we demonstrate a basic method using the scikit-image library.

In [ ]:
# Install scikit-image if needed (Colab/local)# !pip install scikit-imagefrom skimage.filters import threshold_otsudef mask_clouds(image, threshold=None):    # image: numpy array (H, W) or (bands, H, W)    # Use the first band for cloud detection (or use a cloud mask if available)    if image.ndim == 3:        band = image[0]    else:        band = image    if threshold is None:        threshold = threshold_otsu(band)    mask = band < threshold  # clouds are bright, so mask out high values    # Set cloud pixels to 0 (or np.nan if preferred)    if image.ndim == 3:        image[:, ~mask] = 0    else:        image[~mask] = 0    return image

## 6A. Resample All Tiles to Average Resolution (with Cloud Masking)This approach writes each resampled, cloud-masked raster to disk, minimizing RAM usage. Works on both Colab and local machines.

In [ ]:
import shutil# Create a temp directory for resampled filesTEMP_DIR = '/content/temp_resampled' if 'google.colab' in str(get_ipython()) else 'temp_resampled'os.makedirs(TEMP_DIR, exist_ok=True)# Calculate average resolutionall_res = [src.res for src in src_files_to_mosaic]avg_xres = sum([r[0] for r in all_res]) / len(all_res)avg_yres = sum([r[1] for r in all_res]) / len(all_res)avg_res = (avg_xres, avg_yres)print(f"Average resolution: {avg_res}")from rasterio.enums import Resamplingfrom rasterio.warp import calculate_default_transform, reprojectresampled_paths = []for idx, src in enumerate(src_files_to_mosaic):    dst_transform, width, height = calculate_default_transform(        src.crs, src.crs, src.width, src.height, *src.bounds, resolution=avg_res    )    dst_kwargs = src.meta.copy()    dst_kwargs.update({        'height': height,        'width': width,        'transform': dst_transform    })    out_path = os.path.join(TEMP_DIR, f'resampled_{idx}.tif')    with rasterio.open(out_path, 'w', **dst_kwargs) as dst:        for i in range(1, src.count + 1):            data = np.empty((height, width), dtype=src.dtypes[0])            reproject(                source=rasterio.band(src, i),                destination=data,                src_transform=src.transform,                src_crs=src.crs,                dst_transform=dst_transform,                dst_crs=src.crs,                resampling=Resampling.bilinear            )            # Apply cloud masking to each band            data = mask_clouds(data)            dst.write(data, i)    resampled_paths.append(out_path)print(f"Resampled files written to: {TEMP_DIR}")

## 6B. Mosaic Creation (Using Disk-Based Resampled Tiles)Open the resampled files from disk and mosaic them. This step is also memory-efficient.

In [ ]:
# Open resampled files and mosaicresampled_datasets = [rasterio.open(p) for p in resampled_paths]mosaic, out_trans = merge(resampled_datasets)# Show the mosaic (first band)plt.figure(figsize=(10, 10))show(mosaic[0], title="Mosaic - Band 1 (Resampled, Disk-Based, Cloud-Masked)")# Close datasetsfor ds in resampled_datasets:    ds.close()

## 6C. Clean Up Temporary FilesRemove the temporary resampled files and directory to free up disk space.

In [ ]:
# Remove the temp directory and all its filesshutil.rmtree(TEMP_DIR)print(f"Temporary directory {TEMP_DIR} deleted.")

In [ ]:
# Mosaic resampled files directly to disk using gdal_merge.py (RAM-efficient, works in Colab/local)# If running in Colab, install GDAL if not already presenttry:    import subprocess    subprocess.run(['gdal_merge.py', '--help'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)except Exception:    print('Installing GDAL...')    !apt-get install -y gdal-binoutput_path = "cloudless_mosaic.tif"input_files = " ".join(resampled_paths)cmd = f"gdal_merge.py -o {output_path} -of GTiff {input_files}"print("Running:", cmd)subprocess.run(cmd, shell=True, check=True)print(f"Mosaic saved as {output_path}")

In [ ]:
# Copy the generated mosaic GeoTIFF to Google Drive (Colab only)import shutildrive_dest = '/content/drive/My Drive/cloudless_mosaic.tif'shutil.copy('cloudless_mosaic.tif', drive_dest)print(f'Mosaic copied to {drive_dest}')

## 7. Output & Georeferencing ValidationExport the mosaic as a GeoTIFF and verify its spatial accuracy and metadata.

In [ ]:
# Open the output GeoTIFF and print metadatawith rasterio.open('cloudless_mosaic.tif') as src:    print('CRS:', src.crs)    print('Transform:', src.transform)    print('Width:', src.width, 'Height:', src.height)    print('Bounds:', src.bounds)    print('Driver:', src.driver)    print('Count (bands):', src.count)    print('Dtype:', src.dtypes)    # Optionally, plot the mosaic    plt.figure(figsize=(12, 12))    show(src.read(1), title='Final Cloudless Mosaic - Band 1')

## 8. Spatial Accuracy CheckTo verify spatial accuracy, overlay the mosaic bounds on a basemap or compare with known reference points. Here, we print the bounds and show how to overlay with contextily (if you have a web mercator projection).

In [ ]:
# Optional: Overlay bounds on a basemap (requires contextily and pyproj, only for web mercator projection EPSG:3857)# !pip install contextily pyprojimport rasterioimport matplotlib.pyplot as plttry:    import contextily as ctx    from rasterio.warp import transform_bounds    with rasterio.open('cloudless_mosaic.tif') as src:        bounds = src.bounds        crs = src.crs        # Transform bounds to EPSG:3857 for contextily        web_bounds = transform_bounds(crs, 'EPSG:3857', *bounds)        fig, ax = plt.subplots(figsize=(10, 10))        ax.set_xlim(web_bounds[0], web_bounds[2])        ax.set_ylim(web_bounds[1], web_bounds[3])        ctx.add_basemap(ax, crs='EPSG:3857')        # Draw the mosaic bounds        rect = plt.Rectangle((web_bounds[0], web_bounds[1]),                            web_bounds[2] - web_bounds[0],                            web_bounds[3] - web_bounds[1],                            linewidth=2, edgecolor='red', facecolor='none')        ax.add_patch(rect)        plt.title('Mosaic Bounds on Basemap')        plt.show()except ImportError:    print('contextily/pyproj not installed. Skipping basemap overlay.')

## 9. Summary and Next Steps- This notebook demonstrates the workflow for creating a seamless, cloudless raster mosaic from multiple GeoTIFF tiles.- Cloud masking is applied using a simple threshold; for better results, use provided cloud masks or advanced models.- Output GeoTIFF metadata and spatial accuracy are validated.- For submission, ensure the notebook runs end-to-end and produces the required output file.